In [1]:
from rdflib import Graph
import pandas as pd
from rdflib import URIRef,Literal
from rdflib.namespace import RDF,RDFS

In [2]:
#create a copy of a given ttl file


g = Graph()
#g.parse("../fairness_ontology-2.ttl", format='turtle')
g.parse("./FO3_no_ind.ttl", format='turtle')
print(len(g))
g.serialize(destination="tbl.ttl")


if (URIRef("http://fairness_ontology.com/FairnessThroughAwareness"), None, None) in g:
    print("This graph contains triples about FairnessThroughAwareness!")
else:
    print("FairnessThroughAwareness Not found")


741
FairnessThroughAwareness Not found


In [3]:
#load the content of an excel sheet
df = pd.read_excel("../Fairness-Metrics.xlsx", sheet_name="METRICS",header=0, dtype=str)
df.fillna('', inplace=True)

def camelCaseMetric(s):
    temp = s.replace('_', ' ').replace('-', ' ').split('(')[0]
    temp = ' '.join([w.title() if w.islower() else w for w in temp.split()])
    temp=temp.replace(' ', '')
    #res = temp[0].lower() + temp[1:]
    return temp

df['Metric']=df['Metric'].apply(lambda x: camelCaseMetric(x))
#with pd.option_context('display.max_rows', None):
#    print(df['Metric'])
df.head()

,Metric,same as,Definition,Formula,Latex Description,Notion/Metric,Measures,Is measured by,Procedural/Outcome,Individual/Group,...,Domain of Application,Is suitable for,applicable to LLMs,Proposed by,Used by,Equivalent to,Conflicts with,Extends,Other relations,Notes
0,FairnessThroughUnawareness,,An algorithm is fair as long as any protected ...,Remove the sensitive features from the trainin...,An algorithm is fair as long as any protected ...,notion,NONE,NONE,procedural,not applicable,...,NONE,"Classification, text classification",yes,2016_GRGIC-HLACA,NONE,NONE,Any other fairness measure that considers the ...,NONE,NONE,"not actually a measure, might increase bias in..."
1,FairnessThroughAwareness,Individual fairness,An algorithm is fair if it gives similar predi...,"𝐷 (𝑀 (𝑥1), 𝑀 (𝑥2)) ≤ 𝑑 (𝑥1, 𝑥2), ∀𝑥1, 𝑥2 ∈ 𝐸 \...",An algorithm is fair if it gives similar predi...,notion,not applicable,"Total variation norm, Relative l-infinite metr...",outcome,individual,...,"health, law, education",Classification,no,2012_DWORK,"2025_GAO, 2018_YONA, 2020_YUROCHKIN, 2019_LAHOTI",Individual fairness,NONE,NONE,NONE,
2,IndividualFairness,Fairness through awareness,,,,,,,,,...,,,,,,,,,,
3,TotalVariationNorm,,"Given P and Q, probability measures on a finit...",,"Given P and Q, probability measures on a finit...",metric,Fairness through awareness,not applicable,outcome,individual,...,"health, law, education",Classification,no,2012_DWORK,"2025_GAO, 2018_YONA, 2020_YUROCHKIN, 2019_LAHOTI",NONE,NONE,Fairness through awareness,NONE,It is the first poposed metric connected to th...
4,RelativeLInfiniteMetric,,"Given P and Q, probability measures on a finit...",,"Given P and Q, probability measures on a finit...",metric,Fairness through awareness,not applicable,outcome,individual,...,"health, law, education",Classification,no,2012_DWORK,"2025_GAO, 2018_YONA, 2020_YUROCHKIN, 2019_LAHOTI",NONE,NONE,Fairness through awareness,NONE,It is the second poposed metric connected to t...


In [4]:
#ADD THE PAPER IF NOT ALREADY PRESENT
def add_new_scientific_paper(row, prefix, paper_col):
    for paper in row[paper_col].split(','):
        #print("Calling for "+paper)
        paper=paper.strip()
        if paper != 'NONE' and paper != '':
            if (URIRef(prefix+paper), None, None) in g:   #PAPER ALREADY IN THE ONTOLOGY
                print('Paper '+prefix+paper+' already in the ontology')
                pass
            else:
                #print('Adding '+prefix+paper)
                g.add((URIRef(prefix+paper), RDF.type, URIRef("http://www.w3.org/2002/07/owl#NamedIndividual")))
                g.add((URIRef(prefix+paper), RDF.type, URIRef(prefix+"ScientificPaper")))
                g.add((URIRef(prefix+paper), RDFS.label,Literal(paper)))
            
def add_common_elements_for_new_metric_notion(row, metricURI, prefix):
    ## Domain of Application
    if 'education' in row['Domain of Application']:
        g.add((metricURI, URIRef(prefix+"hasBeenAppliedTo"), URIRef(prefix+"Education")))
    if 'law' in row['Domain of Application']:
        g.add((metricURI, URIRef(prefix+"hasBeenAppliedTo"), URIRef(prefix+"Law")))
    if 'census income prediction' in row['Domain of Application'] :
        g.add((metricURI, URIRef(prefix+"hasBeenAppliedTo"), URIRef(prefix+"CensusIncomePrediction")))
    if 'health' in row['Domain of Application']:
        g.add((metricURI, URIRef(prefix+"hasBeenAppliedTo"), URIRef(prefix+"Health")))
    if 'credit risk' in row['Domain of Application']:
        g.add((metricURI, URIRef(prefix+"hasBeenAppliedTo"), URIRef(prefix+"CreditRisk")))
    if 'item recommendation' in row['Domain of Application']:
        g.add((metricURI, URIRef(prefix+"hasBeenAppliedTo"), URIRef(prefix+"ItemRecommendation")))
    if 'chatbot interation' in row['Domain of Application']:
        g.add((metricURI, URIRef(prefix+"hasBeenAppliedTo"), URIRef(prefix+"ChatbotInteraction")))
    
    ## Is suitable for (a ML task)
    if 'Classification' in row['Is suitable for']:
        g.add((metricURI, URIRef(prefix+"isSuitableFor"), URIRef(prefix+"Classification")))
    if 'text classification' in row['Is suitable for']:
        g.add((metricURI, URIRef(prefix+"isSuitableFor"), URIRef(prefix+"TextClassification")))
    if 'masked language modelling' in row['Is suitable for']:
        g.add((metricURI, URIRef(prefix+"isSuitableFor"), URIRef(prefix+"MaskedLanguageModelling")))
    if 'token classification' in row['Is suitable for']:
        g.add((metricURI, URIRef(prefix+"isSuitableFor"), URIRef(prefix+"TokenClassification")))
    
    # is proposed by 
    add_new_scientific_paper(row,prefix,'Proposed by')
    for paper in row['Proposed by'].split(','):
        if paper != 'NONE' and paper != '':
            g.add((metricURI, URIRef(prefix+"isProposedBy"), URIRef(prefix+paper.strip())))
    # is used by
    add_new_scientific_paper(row,prefix,'Used by')
    for paper in row['Used by'].split(','):
        if paper != 'NONE' and paper != '':
            g.add((metricURI, URIRef(prefix+"isUsedBy"), URIRef(prefix+paper.strip())))
    # is equivalent to
    for el in row['Equivalent to'].split(','):
        if el != 'NONE' and el != '':
            g.add((metricURI, URIRef(prefix+"isEquivalentTo"), URIRef(prefix+camelCaseMetric(el.strip()))))
    #extends
    for el in row['Extends'].split(','):
        if el != 'NONE' and el != '':
            g.add((metricURI, URIRef(prefix+"extends"), URIRef(prefix+camelCaseMetric(el.strip()))))
    #notes
    if row['Notes'] != '':
        g.add((metricURI, URIRef("http://www.w3.org/2004/02/skos/core#definition"), Literal(row['Notes'])))

def add_new_notion(metricURI,prefix,row):
    g.add((metricURI, RDF.type, URIRef("http://www.w3.org/2002/07/owl#NamedIndividual")))
    if row['same as'] != '':
        for el in row['same as'].split(','):
            g.add((metricURI, URIRef("http://www.w3.org/2002/07/owl#sameAs"), URIRef(prefix+camelCaseMetric(el.strip()))))
    if row['Latex Description'] != '':
        g.add((metricURI, URIRef(prefix+"scientificArtifactDescription"), Literal(row['Latex Description'])))
    if row['Procedural/Outcome'] == 'procedural':
        g.add((metricURI, RDF.type, URIRef(prefix+"ProceduralFairnessNotion")))
    elif row['Procedural/Outcome'] == 'outcome':
        if row['Causal/Observational'] == 'causal' :
            g.add((metricURI, RDF.type, URIRef(prefix+"CausalFairnessNotion")))
        else:
            g.add((metricURI, RDF.type, URIRef(prefix+"ObservationalFairnessNotion")))
    
        if 'individual' in row['Individual/Group'] :
            g.add((metricURI, RDF.type, URIRef(prefix+"IndividualFairnessNotion")))
        else:
            if 'independence' in row['Independence/Separation/Sufficiency'] :
                g.add((metricURI, RDF.type, URIRef(prefix+"IndependenceFairnessNotion")))
            if 'separation' in row['Independence/Separation/Sufficiency'] :
                g.add((metricURI, RDF.type, URIRef(prefix+"SeparationFairnessNotion")))
            if 'sufficiency' in row['Independence/Separation/Sufficiency'] :
                g.add((metricURI, RDF.type, URIRef(prefix+"SufficiencyFairnessNotion")))
    add_common_elements_for_new_metric_notion(row, metricURI, prefix)
    
def add_new_metric(metricURI,prefix,row):
    g.add((metricURI, RDF.type, URIRef("http://www.w3.org/2002/07/owl#NamedIndividual")))
    #if row['same as'] != '':  #AT THE MOMENT THERE ARE NO IDENTICAL METRICS
    #    g.add((metricURI, URIRef("http://www.w3.org/2002/07/owl#sameAs"), URIRef(prefix+camelCaseMetric(row['same as']))))
    if row['Latex Description'] != '':
        g.add((metricURI, URIRef(prefix+"scientificArtifactDescription"), Literal(row['Latex Description'])))
    if 'calibration' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"CalibrationBasedFairnessMetric")))
    if 'confusion matrix' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"ConfusionMatrixBasedFairnessMetric")))
    if 'distance based' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"DistanceBasedFairnessMetric")))
    if 'item' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"ItemFocusedMetric")))
    if 'user' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"UserFocusedMetric")))
    if 'single' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"SingleGranularityMetric")))
    if 'amortized' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"AmortizedGranularityMetric")))
    if 'treatment' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"TreatmentFocusedMetric")))
    if 'impact' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"ImpactFocusedMetric")))
    if 'parity' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"ParityBasedFairnessMetric")))
    if 'score' in row['Metric class'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"ScoreBasedFairnessMetric")))
    
    if 'Classifier' in row['LLM subcategory'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"ClassifierFairnessMetric")))
    if 'Distribution' in row['LLM subcategory'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"DistributionFairnessMetric")))
    if 'Lexicon' in row['LLM subcategory'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"LexiconFairnessMetric")))
    if 'Masked token' in row['LLM subcategory'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"MaskedTokenFairnessMetric")))
    if 'Pseudo-log likelihood' in row['LLM subcategory'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"PseudoLogLikelihoodFairnessMetric")))
    if 'Sentence embedding' in row['LLM subcategory'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"SentenceEmbeddingFairnessMetric")))
    if 'Word embedding' in row['LLM subcategory'] :
        g.add((metricURI, RDF.type, URIRef(prefix+"WordEmbeddingFairnessMetric")))
    
    add_common_elements_for_new_metric_notion(row, metricURI, prefix)
    


In [5]:

prefix="http://fairness_ontology.com/"

for index, row in df.iterrows():
    metric_name=row['Metric']
    metricURI=URIRef(prefix+metric_name)
    if (metricURI, None, None) in g:   #NOTION/METRIC ALREADY IN THE ONTOLOGY
        print("This graph contains triples about "+str(metricURI))
        pass
    else:
        if row["Notion/Metric"] == "notion":   #NOVEL NOTION
            #print("Notion: "+ str(metricURI)+" Not found")
            add_new_notion(metricURI,prefix,row)
            pass
        elif row["Notion/Metric"] == "metric":   #NOVEL METRIC
            #print("Metric: "+ str(metricURI)+" Not found")
            add_new_metric(metricURI,prefix,row)
            pass

#now that all new metrics and notions have been added, we can link them to each other
for index, row in df.iterrows():
    metric_name=row['Metric']
    metricURI=URIRef(prefix+metric_name)
    if row["Notion/Metric"] == "metric":   #NOVEL METRIC
        for el in row['Measures'].split(','):
            if el != 'NONE' and el != '' and el != 'not applicable':
                g.add((metricURI, URIRef(prefix+"measures"), URIRef(prefix+camelCaseMetric(el.strip()))))
    if row["Notion/Metric"] == "notion":   #NOVEL NOTION
        for el in row['Is measured by'].split(','):
            if el != 'NONE' and el != '' and el != 'not applicable':
                g.add((metricURI, URIRef(prefix+"isMeasuredBy"), URIRef(prefix+camelCaseMetric(el.strip()))))





Paper http://fairness_ontology.com/2012_DWORK already in the ontology
Paper http://fairness_ontology.com/2025_GAO already in the ontology
Paper http://fairness_ontology.com/2018_YONA already in the ontology
Paper http://fairness_ontology.com/2020_YUROCHKIN already in the ontology
Paper http://fairness_ontology.com/2019_LAHOTI already in the ontology
Paper http://fairness_ontology.com/2012_DWORK already in the ontology
Paper http://fairness_ontology.com/2025_GAO already in the ontology
Paper http://fairness_ontology.com/2018_YONA already in the ontology
Paper http://fairness_ontology.com/2020_YUROCHKIN already in the ontology
Paper http://fairness_ontology.com/2019_LAHOTI already in the ontology
Paper http://fairness_ontology.com/2025_WALLER already in the ontology
Paper http://fairness_ontology.com/2025_WALLER already in the ontology
Paper http://fairness_ontology.com/2025_WALLER already in the ontology
Paper http://fairness_ontology.com/2025_WALLER already in the ontology
Paper http:/

In [6]:
# LOADING THE MITIGATION TECHNIQUES:
df = pd.read_excel("../Fairness-Metrics.xlsx", sheet_name="(Ont) Mitigation",header=0, dtype=str)
df.fillna('', inplace=True)

for index, row in df.iterrows():
    mtech_name=row['Mitigation Technique']
    mtechURI=URIRef(prefix+camelCaseMetric(mtech_name))
        
    if (mtechURI, None, None) in g:   #MIT. TECH. ALREADY IN THE ONTOLOGY
        print("This graph alresy contains triples about "+str(mtechURI))
        pass
    else:
        # Add the new mitigation technique to the graph
        g.add((mtechURI, RDF.type, URIRef("http://www.w3.org/2002/07/owl#NamedIndividual")))
        for category in row["A-J Category"].split(','):
            g.add((mtechURI, RDF.type, URIRef(prefix+category.strip())))
        if row['Description'] != '':
            g.add((mtechURI, URIRef(prefix+"scientificArtifactDescription"), Literal(row['Description'])))
        add_new_scientific_paper(prefix=prefix,row=row,paper_col="Source")
        for paper in row['Source'].split(','):
            if paper != 'NONE' and paper != '':
                g.add((mtechURI, URIRef(prefix+"isProposedBy"), URIRef(prefix+paper.strip())))
     

g.serialize(destination="tbl.ttl")


Paper http://fairness_ontology.com/2018_FARNADI already in the ontology
Paper http://fairness_ontology.com/2017_YAO already in the ontology
Paper http://fairness_ontology.com/2019_RASTEGARPANAH already in the ontology
Paper http://fairness_ontology.com/2018_STECK already in the ontology
Paper http://fairness_ontology.com/2020_ZHU already in the ontology
Paper http://fairness_ontology.com/2018_KAMISHIMA already in the ontology
Paper http://fairness_ontology.com/2020_PATRO already in the ontology
Paper http://fairness_ontology.com/2021_MANSOURY already in the ontology
Paper http://fairness_ontology.com/2020_WAN already in the ontology
Paper http://fairness_ontology.com/2019_MAUDSLAY already in the ontology
Paper http://fairness_ontology.com/2020_GEHMAN already in the ontology
Paper http://fairness_ontology.com/2019_SHENG already in the ontology
Paper http://fairness_ontology.com/2019_CELISa already in the ontology
Paper http://fairness_ontology.com/2021_GE already in the ontology
Paper h

<Graph identifier=Nacad89680e0c4e18a461819feb6ecb0d (<class 'rdflib.graph.Graph'>)>